In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors # Use this is for creating a cursor-interactive plot with "%matplotlib notebook"
from sklearn.decomposition import NMF # Use this for training Non-negative Matrix Factorization
from sklearn.utils.extmath import randomized_svd # Use this for training Singular Value Decomposition
from sklearn.manifold import TSNE # Use this for training t-sne manifolding

In [2]:
dir = './최종데이터/'
#df_product = pd.read_csv(dir + 'basic_data_edit.csv', usecols=['00.상품_URL','00.상품코드','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음'],encoding='cp949')
df_product = pd.read_csv(dir + 'basic_data_img.csv', usecols=['00.상품코드','00.상품_URL','00.이미지_URL','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음'],encoding='cp949')
df_review = pd.read_csv(dir + 'total_review.csv', usecols=['code','user','type','tone','problem','rating','feature','review','total_rating'],encoding='cp949')
print('done')

done


In [3]:
df_review

,code,user,type,tone,problem,rating,feature,review,total_rating
0,A000000147339,요다,건성,웜톤,민감성 | 아토피 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",개인적으로 성분으로 마케팅하는 제품들을 그렇게 좋아하진 않습니다. 특히 토너는 순한...,5.0
1,A000000147339,올리브양,복합성,쿨톤,모공 | 민감성 | 주름 | 탄력 | 트러블 | 홍조,5.0,"('건성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",복합성 피부에 많이 민감한 피부입니다. 요즘 봄 바람에 아침 저녁으로 기온차가 심해...,5.0
2,A000000147339,뚜비이,건성,쿨톤,민감성 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",지난번 올영체험단으로 제공받아 사용해보고 너무 좋아서 다시 구매했습니당!\r\n저는...,5.0
3,A000000147339,시래우,복합성,겨울쿨톤,다크서클 | 모공 | 미백 | 민감성 | 블랙헤드 | 아토피 | 잡티 | 주름 | ...,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",날씨 더워지면 광대가 뜨거워지면서 내 얼굴이 호빵맨이 되어가는 것 같고 좀 더 더워...,5.0
4,A000000147339,효지니,복합성,쿨톤,민감성 | 블랙헤드 | 아토피 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",요즘에 피부가 너무 나빠져서 잘 맞는 화장품 찾아볼려고 이것저것 구매를 해봤어요! ...,5.0
...,...,...,...,...,...,...,...,...,...
16500,A000000148095,겨울별,지성,쿨톤,각질 | 미백 | 블랙헤드 | 잡티 | 주름 | 트러블,5.0,"('지성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",봄철에딱인 속건조토너 흑미탄탄토너를 일주일정도 써보았습니다. 뷰티유튜버가 왜추천을했...,3.5
16501,A000000148095,moon219,복합성,쿨톤,모공 | 트러블,5.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",제품이 순하기도 하고 성분이 좋아서 그런지 토너팩을 매일같이 하니까 정말 많이 진정...,5.0
16502,A000000148095,리파리해파리파리,건성,웜톤,각질 | 모공 | 미백 | 블랙헤드 | 잡티 | 주름 | 탄력 | 트러블,4.0,"('복합성에 좋아요 | 주름/미백에 좋아요 | 자극없이 순해요',)",저는 올리브영체험단을 몇번하면서\r\n항상 제품과 리뷰작성바란다는 안내문만 받아보다...,3.0
16503,A000000148095,luv9,건성,웜톤,각질 | 민감성 | 주름 | 홍조,4.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",체험단 신청해서 받아보게 되었는데! 직접 손편지로 리뷰작성안내 해주신 브랜드는 처음...,3.5


In [4]:
#2개 이상의 리뷰를 단 유저들만 추출
user_review_count = df_review['user'].value_counts()
user_review_count = pd.DataFrame(user_review_count)
user_review_count = user_review_count.reset_index()
user_review_count.columns = ['user','count']

df_review_count = pd.merge(df_review,user_review_count,on='user',how='left')
df_review_count = df_review_count[df_review_count['count']>=2]
df_review_count

,code,user,type,tone,problem,rating,feature,review,total_rating,count
0,A000000147339,요다,건성,웜톤,민감성 | 아토피 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",개인적으로 성분으로 마케팅하는 제품들을 그렇게 좋아하진 않습니다. 특히 토너는 순한...,5.0,14
1,A000000147339,올리브양,복합성,쿨톤,모공 | 민감성 | 주름 | 탄력 | 트러블 | 홍조,5.0,"('건성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",복합성 피부에 많이 민감한 피부입니다. 요즘 봄 바람에 아침 저녁으로 기온차가 심해...,5.0,13
2,A000000147339,뚜비이,건성,쿨톤,민감성 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",지난번 올영체험단으로 제공받아 사용해보고 너무 좋아서 다시 구매했습니당!\r\n저는...,5.0,6
3,A000000147339,시래우,복합성,겨울쿨톤,다크서클 | 모공 | 미백 | 민감성 | 블랙헤드 | 아토피 | 잡티 | 주름 | ...,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",날씨 더워지면 광대가 뜨거워지면서 내 얼굴이 호빵맨이 되어가는 것 같고 좀 더 더워...,5.0,13
4,A000000147339,효지니,복합성,쿨톤,민감성 | 블랙헤드 | 아토피 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",요즘에 피부가 너무 나빠져서 잘 맞는 화장품 찾아볼려고 이것저것 구매를 해봤어요! ...,5.0,10
...,...,...,...,...,...,...,...,...,...,...
16500,A000000148095,겨울별,지성,쿨톤,각질 | 미백 | 블랙헤드 | 잡티 | 주름 | 트러블,5.0,"('지성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",봄철에딱인 속건조토너 흑미탄탄토너를 일주일정도 써보았습니다. 뷰티유튜버가 왜추천을했...,3.5,5
16501,A000000148095,moon219,복합성,쿨톤,모공 | 트러블,5.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",제품이 순하기도 하고 성분이 좋아서 그런지 토너팩을 매일같이 하니까 정말 많이 진정...,5.0,6
16502,A000000148095,리파리해파리파리,건성,웜톤,각질 | 모공 | 미백 | 블랙헤드 | 잡티 | 주름 | 탄력 | 트러블,4.0,"('복합성에 좋아요 | 주름/미백에 좋아요 | 자극없이 순해요',)",저는 올리브영체험단을 몇번하면서\r\n항상 제품과 리뷰작성바란다는 안내문만 받아보다...,3.0,27
16503,A000000148095,luv9,건성,웜톤,각질 | 민감성 | 주름 | 홍조,4.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",체험단 신청해서 받아보게 되었는데! 직접 손편지로 리뷰작성안내 해주신 브랜드는 처음...,3.5,10


In [5]:
#사용자가 입력한 속성과 가장 비슷한 기존 유저를 뽑는 작업
df_user_feature = df_review_count[['user','type','tone','problem']]
df_user_feature = df_user_feature.drop_duplicates(['user'])

#user의 통합 feature를 담을 데이터 프레임
df_feature = pd.DataFrame(columns = ['user','feature'])

for index in df_user_feature.index:
    user = df_user_feature.loc[index]['user']
    problems = df_user_feature.loc[index]['problem']
    problems_list = problems.split('|')
    
    problem = ''
    for i in problems_list:
        i = i.replace(" ","")
        problem += (i+' ')
    
    feature = str(df_user_feature.loc[index]['type']) +' '+str(df_user_feature.loc[index]['tone']) + ' '+problem
    
    df_feature.loc[str(index)] = [user,feature]
    
#사용자가 입력한 정보를 df_feature 맨 아래에 추가  
df_feature
df_feature.loc[str(index)] = ('input','복합성 쿨톤 민감성 블랙헤드 아토피 잡티 트러블')
df_feature = df_feature.reset_index(drop=True)
print(df_feature)

        user                                            feature
0         요다                                 건성 웜톤 민감성 아토피 트러블 
1       올리브양                        복합성 쿨톤 모공 민감성 주름 탄력 트러블 홍조 
2        뚜비이                                  건성 쿨톤 민감성 잡티 트러블 
3        시래우  복합성 겨울쿨톤 다크서클 모공 미백 민감성 블랙헤드 아토피 잡티 주름 탄력 트러블 홍조 
4        효지니                        복합성 쿨톤 민감성 블랙헤드 아토피 잡티 트러블 
...      ...                                                ...
3161  dawoon                                    건성 웜톤 모공 잡티 탄력 
3162   고구맛탱이                                        건성 쿨톤 블랙헤드 
3163    에주에주                                       지성 웜톤 모공 미백 
3164   grape        복합성 쿨톤 각질 모공 미백 민감성 블랙헤드 잡티 주름 탄력 트러블 피지과다 
3165   input                         복합성 쿨톤 민감성 블랙헤드 아토피 잡티 트러블

[3166 rows x 2 columns]


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

counter_vector = CountVectorizer(ngram_range=(1,3))
c_vector_features = counter_vector.fit_transform(df_feature['feature'])
c_vector_features.shape

(3166, 883)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_feature = cosine_similarity(c_vector_features,c_vector_features).argsort()[:,::-1]
print(similarity_feature)
print(similarity_feature.shape)

[[   0  416  934 ... 1178 1182 1582]
 [   1 1361  538 ... 2420 1229 1015]
 [ 733    2 2192 ... 2467 1343 2324]
 ...
 [ 817 3163 1951 ...  611 2101 3165]
 [1550 3164 2612 ...  980  734 2855]
 [3165    4 1263 ... 1854 2581 2739]]
(3166, 3166)


In [8]:
def recommend_user_list(df_feature, user , top=3):
    #특정 유저 뽑아내기
    target_feature_index = df_feature[df_feature['user'] == user].index.values

    #타켓제품과 비슷한 코사인 유사도값
    sim_index = similarity_feature[int(target_feature_index),:top].reshape(-1)
    #본인제외
    sim_index = sim_index[sim_index!=target_feature_index]
    
    #추천 결과 새로운 df 생성, 평균평점(score)으로 정렬
    result = df_feature.iloc[sim_index]
    
    return result

In [9]:
df_result = recommend_user_list(df_feature, user='input')
similar_user = df_result['user']
similar_user = similar_user.reset_index(drop=True)
similar_user = similar_user[0]
similar_user

'효지니'

In [10]:
# 고유 사용자, 고유 제품 갯수 확인
n_users = len(df_review_count['user'].unique())
n_product = len(df_review_count['code'].unique())

n_users, n_product # 610 명의 사용자가 9724개의 영화에 평점을 매김을 확인

(3166, 368)

In [11]:
name_list = df_review_count['user'].unique()
name_list = pd.Series(name_list)
name_list

0           요다
1         올리브양
2          뚜비이
3          시래우
4          효지니
         ...  
3161    dawoon
3162     고구맛탱이
3163      에주에주
3164     grape
3165       겨울별
Length: 3166, dtype: object

In [12]:
name_list = df_review_count['code'].unique()
name_list = pd.Series(name_list)
name_list

0      A000000147339
1      A000000137180
2      A000000141704
3      A000000104007
4      A000000146589
           ...      
363    A000000122212
364    A000000148253
365    A000000122094
366    A000000121274
367    A000000148095
Length: 368, dtype: object

In [13]:
A = df_review_count.pivot_table(index = 'code', columns = 'user',values = 'total_rating')
A = A.copy().fillna(0)
print(A)

user           02k  0321  0eun   0교   11  11월의온조  12최정  16살  17호빋베브봄  1등급  \
code                                                                        
A000000002839  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000004434  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005060  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005766  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005767  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
...            ...   ...   ...  ...  ...     ...   ...  ...      ...  ...   
A000000163655  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000163896  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000164211  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000165599  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000166043  0.0   0.0   0.0  0.0  4.0     0.0   0.0  0.0      0.0  0.0   

In [14]:
A_transpose = A.transpose()
user_based_collabor = cosine_similarity(A_transpose)
user_based_collabor = pd.DataFrame(data = user_based_collabor, index = A_transpose.index, columns = A_transpose.index)
user_based_collabor

user,02k,0321,0eun,0교,11,11월의온조,12최정,16살,17호빋베브봄,1등급,...,힇힇,히덩,히야,히요미,히히만두,히히하하,히히히하,힝,힝꾸꾸,힝소
user,,,,,,,,,,,,,,,,,,,,,
02k,1.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0321,0.0,1.0000,0.0,0.5547,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0eun,0.0,0.0000,1.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.319624,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0교,0.0,0.5547,0.0,1.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0000,0.0,0.0000,1.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
히히하하,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
히히히하,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
힝,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
#특정 사용자와 가장 비슷한 유저 추출
#index = name_list[name_list == similar_user].index[0]

def get_user_based_collabor(user):
    return user_based_collabor[user].sort_values(ascending=False)[:15]
similar_user_list = get_user_based_collabor(similar_user)
similar_user_list=similar_user_list[1:15]
similar_user_list = similar_user_list.index
similar_user_list

Index(['리뷰리뷰ㅎㅅㅎ', '돈쓰는로라', 'BONALOG', '또잉또잉ㅇ', '1등급', '마포구핑킹가위', 'Songi',
       '하이00', '김지수', 'gumg01', '당근당긍', 'ha느리', 'ahammt', '블루걸'],
      dtype='object', name='user')

In [20]:
print('-----가장 비슷한 유저 Top10-----')
for i in similar_user_list:
    print(i)

-----가장 비슷한 유저 Top10-----
리뷰리뷰ㅎㅅㅎ
돈쓰는로라
BONALOG
또잉또잉ㅇ
1등급
마포구핑킹가위
Songi
하이00
김지수
gumg01
당근당긍
ha느리
ahammt
블루걸


In [17]:
code_list = []
for r1 in similar_user_list:
    max_rating = A[r1].max()
    cos_id = A[A[r1]==max_rating].index  
    code_list.append(cos_id)
code_list

[Index(['A000000148391', 'A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000147339', 'A000000148391', 'A000000151951', 'A000000152047'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000150946'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A000000147339', 'A000000148391'], dtype='object', name='code'),
 Index(['A000000150947', 'A000000159182'], dtype='object', name='code'),
 Index(['A0000001

In [210]:
final_code_list = []

for codes in code_list:
    for code in codes:
        final_code_list.append(code)
final_code_set = set(final_code_list)
final_code_list = list(final_code_set)
final_code_list

['A000000150946',
 'A000000150947',
 'A000000147339',
 'A000000159182',
 'A000000151951',
 'A000000148391',
 'A000000152047']

In [211]:
df_product
df_product = df_product.drop_duplicates(['00.상품코드'])
df_product = df_product.drop_duplicates(['02.상품명'])

In [212]:
result_dict={}
products_dict = {}
for index, code in enumerate(final_code_list):
    print(code)
    product_dict = {}
    product_dict['productURL'] = str(df_product[df_product['00.상품코드']==code]['00.상품_URL'].item())
    product_dict['imageURL'] = str(df_product[df_product['00.상품코드']==code]['00.이미지_URL'].item())
    product_dict['brand'] = str(df_product[df_product['00.상품코드']==code]['01.브랜드'].item())
    product_dict['productName'] = str(df_product[df_product['00.상품코드']==code]['02.상품명'].item())
    product_dict['price'] = int(df_product[df_product['00.상품코드']==code]['03.가격'].item())
    products_dict[index] = product_dict
    if index == 4:
        break
    
    
result_dict['CF'] = products_dict
print(result_dict)

A000000150946
A000000150947
A000000147339
A000000159182
A000000151951
{'CF': {0: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000150946&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode', 'imageURL': 'https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0015/A00000015094608ko.jpg?l=ko', 'brand': '넘버즈인', 'productName': '넘버즈인 2번 메이크업 찰떡 토너 200ml', 'price': 22000}, 1: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000150947&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode', 'imageURL': 'https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0015/A00000015094714ko.jpg?l=ko', 'brand': '넘버즈인', 'productName': '넘버즈인 3번 결광가득 에센스 토너 200ml', 'price': 22000}, 2: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000147339&dispCatNo=100000100010008&trackingCd=Cat1000001000100